In [1]:
%matplotlib notebook
import numpy as np
import scipy.linalg as la
import mpl_toolkits.mplot3d
import matplotlib.pyplot as plt

from scipy.spatial import cKDTree

import scipy.sparse as sp
from scipy.sparse.linalg import spsolve, lsqr

from numpy.linalg import cond, norm, inv, lstsq

from scipy.optimize import minimize_scalar

from poly_basis import *
from spherepts import *
from rbf import *

from laplacebeltrami import *

from math import factorial as fac

In [5]:
n = 200
eps = None
k = 21
rbf_obj = rbf_dict['multiquadric']
# Choose solution and forcing fuction
solution_index = 10

nodes = gen_spiral_nodes(n)
normals = nodes

rbf = rbf_obj['rbf']
zeta  = rbf_obj['zeta']
d2rbf = rbf_obj['d2rbf']
Lrbf = lambda r,eps: 1*zeta(r,eps) + d2rbf(r,eps)

sol_deg = sphere_harm_degs[solution_index]
exact = lambda x: sphere_harm[solution_index](*x)*-sol_deg*(sol_deg+1)
forcing = lambda x: sphere_harm[solution_index](*x)
print('Harmonic degree: %d' % sol_deg)

Harmonic degree: 3


# Shakar-Wright Method

In [108]:
def grad_poly(nodes, projectors, deg):
    n = len(nodes)
    x = nodes[:,0]
    y = nodes[:,1]
    z = nodes[:,2]
    cols = fac(deg+3)//(fac(deg)*fac(3))
    P = np.zeros((n, cols))
    rhs_dx = np.zeros((cols, n))
    rhs_dy = np.zeros((cols, n))
    rhs_dz = np.zeros((cols, n))
    rhs_x = np.zeros((cols, n))
    rhs_y = np.zeros((cols, n))
    rhs_z = np.zeros((cols, n))
    i = 0
    for d in range(deg+1):
        #x^a y^b z^c with a+b+c = d
        for a in range(d,-1, -1):
            for b in range(d-a, -1, -1):
                c = d-a-b
                P[:,i] = x**a * y**b * z**c
                if a == 0:
                    rhs_dx[i] = 0
                else:
                    rhs_dx[i] = a * x**(a-1) * y**b * z**c
                if b == 0:
                    rhs_dy[i] = 0
                else:
                    rhs_dy[i] = b * x**a * y**(b-1) * z**c
                if c == 0:
                    rhs_dz[i] = 0
                else:
                    rhs_dz[i] = c * x**a * y**b * z**(c-1)
                i += 1
    for i, p in enumerate(projectors):
        rhs_x[:,i] = p[0,0]*rhs_dx[:,i] + p[1,0]*rhs_dy[:,i] + p[2,0]*rhs_dz[:,i]
        rhs_y[:,i] = p[0,1]*rhs_dx[:,i] + p[1,1]*rhs_dy[:,i] + p[2,1]*rhs_dz[:,i]
        rhs_z[:,i] = p[0,2]*rhs_dx[:,i] + p[1,2]*rhs_dy[:,i] + p[2,2]*rhs_dz[:,i]
    return P, rhs_x, rhs_y, rhs_z

def grad_rbf_outer(nodes, centers, zeta, epsilon):
    n_len = len(nodes)
    c_len = len(centers)
    r = dist_outer(nodes, centers)[:,:,np.newaxis]
    xs = (np.array(nodes).reshape((1,n_len,3)) - np.array(centers).reshape((c_len,1,3)))
    return zeta(r, epsilon) * xs

def SWM(nodes, normals, rbf_obj=rbf_dict['multiquadric'], epsilon=None, 
        stencil_size=15, poly_deg=None, poly_type='s'):
    n = len(nodes)
    k = stencil_size
    rbf = rbf_obj['rbf']
    zeta = rbf_obj['zeta']
    
    weights = np.zeros((n, stencil_size))
    row_index = [r for r in range(n) for c in range(stencil_size)]
    col_index = np.zeros((n, stencil_size))
    
    tree = cKDTree(np.array(nodes))
    projectors = [np.eye(3) - np.outer(node, node) for node in normals]
    
    for i, node in enumerate(nodes):
        stencil = tree.query(nodes[i], k)[1]
        col_index[i] = stencil
        nn = np.array([nodes[i] for i in stencil])
        nn_proj = np.array([projectors[i] for i in stencil])
        # center stencil
        nn -= nn[0]
        # scale stencil
        scale = np.max(np.abs(nn))
        nn /= scale        
        
        if poly_deg is None:
            P = None
        else:
            if poly_type == 'p':
                P, rhs_x, rhs_y, rhs_z = grad_poly(nn, nn_proj, poly_deg)
            elif poly_type == 's':
                P, rhs_x, rhs_y, rhs_z = gen_sphere_harm_basis(poly_deg, nn, nn_proj)
            rhs_x /= scale
            rhs_y /= scale
            rhs_z /= scale
            
        #return P
        
        dist_mat = dist_outer(nn,nn)
        #print(cond(rbf(dist_mat, 1)))
        if i==0 and epsilon is None and rbf_obj['shape']:
            if P is None or cond(P)>10**14:
                epsilon = optimize_eps(rbf, dist_mat, P=None, target_cond=10**12)
            else:
                epsilon = optimize_eps(rbf, dist_mat, P=P, target_cond=10**12)
            print('epsilon set: %g' % epsilon)
            
        A = rbf(dist_mat, epsilon)
        rhsAs = np.matmul(nn_proj, grad_rbf_outer(nn, nn, zeta, epsilon).reshape(
                (stencil_size,stencil_size,3,1))).reshape((stencil_size,stencil_size,3))
        rhsAs /= scale
        
        # For testing
        return A, P, rhsAs[:,:,0], rhs_x, dist_mat
        
        if P is None:
            rhs = rhsAs[:,:,0] # only the x coordinates
            weights_grad = la.solve(A, rhs).T
            weights[i] = (weights_grad@weights_grad)[0]

            rhs = rhsAs[:,:,1] # only the y coordinates
            weights_grad = la.solve(A, rhs).T
            weights[i] += (weights_grad@weights_grad)[0]

            rhs = rhsAs[:,:,2] # only the z coordinates
            weights_grad = la.solve(A, rhs)[:stencil_size,:].T
            weights[i] += (weights_grad@weights_grad)[0]
        else:
            schur = True
            if schur:
                weights_grad = schur_solve(A, P, rhsAs[:,:,0], rhs_x)[0]
                weights[i] = (weights_grad@weights_grad)[0]
                weights_grad = schur_solve(A, P, rhsAs[:,:,1], rhs_y)[0]
                weights[i] += (weights_grad@weights_grad)[0]
                weights_grad = schur_solve(A, P, rhsAs[:,:,2], rhs_z)[0]
                weights[i] += (weights_grad@weights_grad)[0]
            else:
                num_basis = P.shape[1]

                AP = np.block([[A,P],[P.T, np.zeros((num_basis,num_basis))]])
                grad_rbf = []
                rhsA = rhsAs[:,:,0] # only the x coordinates
                rhs = np.block([[rhsA],
                               [rhs_x]])
                weights_grad = la.solve(AP, rhs)[:stencil_size,:].T
                weights[i] = (weights_grad@weights_grad)[0]

                rhsA = rhsAs[:,:,1] # only the y coordinates
                rhs = np.block([[rhsA],
                               [rhs_y]])
                weights_grad = la.solve(AP, rhs)[:stencil_size,:].T
                weights[i] += (weights_grad@weights_grad)[0]

                rhsA = rhsAs[:,:,2] # only the z coordinates
                rhs = np.block([[rhsA],
                                [rhs_z]])
                weights_grad = la.solve(AP, rhs)[:stencil_size,:].T
                weights[i] += (weights_grad@weights_grad)[0]

    C = sp.csc_matrix((weights.ravel(), (row_index, col_index.ravel())),shape=(n,n))
    return C

In [109]:
C = SWM(nodes, normals, rbf_obj, eps, 50, poly_deg=1, poly_type='p')
fs = np.array([forcing(node) for node in nodes])
ds = C @ fs
es = np.array([exact(node) for node in nodes])
print(la.norm(ds - es)/la.norm(es))

epsilon set: 0.371308


ValueError: setting an array element with a sequence.

In [110]:
A, P, f, g, dist_mat = SWM(nodes, normals, rbf_obj, None, 50, poly_deg=1, poly_type='p')
print(cond(A))
print(cond(P))

epsilon set: 0.371308
983665812826.5153
6.424753161546076


In [111]:
for x in [A, P, f, g, dist_mat]:
    print(x.shape)

(50, 50)
(50, 4)
(50, 50)
(4, 50)
(50, 50)


In [112]:
def schur_solve(A, P, f, g):
#     U, S, V = la.svd(P, full_matrices=False)
#     num = np.sum( S > 1e-12)
#     print(num, U.shape, S.shape, V.shape)
#     U = U[:,:num]
#     S = S[:num]
#     V = V[:num, :]
#     B = P.T @ la.solve(A, P)
    #lam = lstsq(B, P.T@la.solve(A,f) - g, rcond=10**-14)[0]
    #lam = la.solve(P.T @ la.solve(A, (U*S)@V), P.T @ la.solve(A,f) - g)
    lam = la.solve(P.T @ la.solve(A, P), P.T @ la.solve(A,f) - g)
    w = la.solve(A, f- P@lam)
    return w, lam

In [113]:
w, lam = schur_solve(A, P, f, g)
print(w)

[[ 8.81473330e-01  3.90077802e+00 -4.34399158e+00 ... -1.07047887e+01
   5.79676233e+00  6.07342463e+00]
 [-3.44722250e+00 -1.53686825e+00  2.25467035e+00 ...  4.90526454e+00
  -1.09893726e+01 -5.70596124e+00]
 [ 1.70238995e+00 -1.40673924e+00  4.86862691e-01 ...  8.57405998e+00
  -7.07847837e-01  6.51362462e+00]
 ...
 [-1.54623800e-02 -3.84693266e-04 -6.43168072e-03 ...  1.34654442e+00
   5.49205244e-02 -1.85435037e-01]
 [ 5.05468688e-02  3.15839370e-02  1.76594089e-02 ...  6.10420413e-01
   5.07774472e+00 -1.15434772e-01]
 [-7.11522553e-03 -3.06438031e-02 -1.49919087e-02 ... -2.78953408e-01
   4.17168127e-01 -4.10500284e+00]]


In [121]:
print(np.max(np.abs(P.T @ w - g)))
print(np.max(np.abs(A@w, f - P@lam)))

1.1214362771738706e-12
0.15487210407210547


In [114]:
numP = P.shape[1]
AP = np.block([[A, P],[P.T, np.zeros((numP,numP))]])
fg = np.block([[f],[g]])
wlam = np.block([[w],[lam]])
np.allclose(AP @ wlam, fg)
print(AP @ wlam - fg)

[[ 1.24956947e-16  6.59194921e-16 -2.35922393e-16 ... -7.70217223e-16
   1.33226763e-15 -2.62290190e-15]
 [ 2.01227923e-16 -5.60995405e-17  1.59594560e-16 ...  4.95437025e-15
  -2.77555756e-17 -5.93969318e-15]
 [-5.13478149e-16  2.24820162e-15  1.43003777e-15 ... -2.25687524e-15
  -1.33018596e-14  1.07136522e-14]
 ...
 [-1.94066985e-13 -3.08642001e-14  1.06137321e-13 ...  3.10862447e-15
  -1.70585768e-13  5.52891066e-14]
 [ 8.42060796e-14  7.84650123e-14  2.86073248e-14 ...  3.47499807e-14
  -3.91908728e-14  6.38378239e-15]
 [ 2.13926099e-14 -2.10526041e-13 -3.51718654e-13 ...  3.88161725e-13
  -2.69839706e-13 -3.33788552e-13]]


In [115]:
U, S, V = la.svd(P, full_matrices=False)
num = np.sum( S > 1e-12)
U = U[:,:num]
S = S[:num]
V = V[:num, :]
np.allclose((U*S) @ V, P)
print((U*S) @ V - P)

[[-5.55111512e-16 -1.87350135e-16 -3.46944695e-18 -1.11022302e-16]
 [-4.44089210e-16 -1.11022302e-16  8.32667268e-17 -6.93889390e-17]
 [-5.55111512e-16 -3.05311332e-16  3.33066907e-16 -1.38777878e-16]
 [-4.44089210e-16 -3.88578059e-16 -5.55111512e-17 -4.16333634e-17]
 [-2.22044605e-16 -1.66533454e-16  5.55111512e-17 -1.80411242e-16]
 [-4.44089210e-16 -1.04083409e-16  5.55111512e-17 -1.52655666e-16]
 [-3.33066907e-16  0.00000000e+00  1.24900090e-16 -6.93889390e-17]
 [-4.44089210e-16 -2.49800181e-16  0.00000000e+00 -8.32667268e-17]
 [-4.44089210e-16 -2.22044605e-16 -5.55111512e-17 -5.55111512e-17]
 [-4.44089210e-16 -4.99600361e-16 -1.11022302e-16 -1.38777878e-17]
 [-4.44089210e-16 -3.88578059e-16 -5.55111512e-17 -1.11022302e-16]
 [-3.33066907e-16  5.55111512e-17  1.66533454e-16 -5.55111512e-17]
 [-3.33066907e-16  1.11022302e-16  1.11022302e-16 -8.32667268e-17]
 [-4.44089210e-16 -5.55111512e-17  1.11022302e-16 -2.77555756e-17]
 [-5.55111512e-16 -4.44089210e-16 -1.11022302e-16  5.55111512e

# Tangent Plane Method

In [ ]:
C = TPM(nodes, normals, rbf_obj, eps, k)
fs = np.array([forcing(node) for node in nodes])
ds = C @ fs
es = np.array([exact(node) for node in nodes])
print(la.norm(ds - es)/la.norm(es))

In [ ]:
rbf_obj = rbf_dict['ninth degree PHS']
C = TPM(nodes, normals, rbf_obj, 1, k, poly_deg = 3)
fs = np.array([forcing(node) for node in nodes])
ds = C @ fs
es = np.array([exact(node) for node in nodes])
print(la.norm(ds - es)/la.norm(es))

In [ ]:
deg = 4
for a in range(deg,-1, -1):
    for b in range(deg-a, -1, -1):
        print(a, b, deg-a-b)

In [ ]:
from math import factorial as fac

In [ ]:
fac(4+3)/(fac(4)*fac(3))

In [ ]:
nodes = np.array([[7,2,3],[7,2,3]])

In [ ]:
P, rhs_x, rhs_y, rhs_z = grad_poly(nodes, 3)

In [ ]:
print(P)

In [ ]:
print(rhs_x)

In [ ]:
3*49

In [ ]:
node = nodes[0]

In [ ]:
node

In [ ]:
nn = nodes[:5]

In [ ]:
nn

In [ ]:
nn - node

In [ ]:
nn[1] - node